<h2>Capstone Project_Week 4/5: Let’s do ANY CITY in 3 days!

### 1. Importing the libraries

In [422]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_rows",50, "display.max_columns", None, "display.max_colwidth", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### 2. Determine geographic coordinates of target city

In [423]:
address = 'Jakarta, Indonesia'

geolocator = Nominatim(user_agent="city_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinates of Jakarta, Indonesia are -6.1753942, 106.827183.


### 3. Retrieve venues in target city from Foursquare.com

The objective of this project is to generate a unique itinerary for each of the 3 days of the trip to the target city.

Each day should include the following 5 items:
1. An area with many breakfast venues ("breakfast area")
2. One of the 3 top rated museums ("top museum")
3. One of the 3 top rated sights ("top sight")
4. An area with many restaurants ("dining area")
5. An areas with many bars and clubs ("nightlife area")


#### 3.1 Search for areas with top rated breakfast venues

##### 3.1.1 Retrieve all breakfast venues of target city as a dataframe

In [424]:
# Foursquare.com credentials and parameters
CLIENT_ID = 'XYZ' # your Foursquare ID
CLIENT_SECRET = 'XYZ' # your Foursquare Secret
VERSION = '20201110' # Foursquare API version

categoryId='4bf58dd8d48988d179941735,4bf58dd8d48988d143941735,4bf58dd8d48988d16d941735,4bf58dd8d48988d1e0931735,4bf58dd8d48988d147941735,4bf58dd8d48988d148941735'

#Bagel Shop
#4bf58dd8d48988d179941735
#Breakfast Spot
#4bf58dd8d48988d143941735
#Café
#4bf58dd8d48988d16d941735
#Coffee Shop
#4bf58dd8d48988d1e0931735
#Diner
#4bf58dd8d48988d147941735
#Donut Shop
#4bf58dd8d48988d148941735

breakfast_category_name_exclude=[]

radius=5000
LIMIT = 100 # A default Foursquare API limit value

# create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            latitude, 
            longitude, 
            VERSION, 
            radius,
            LIMIT,
            categoryId)
            
# make the GET request
breakfast_json = requests.get(url).json()['response']['venues']
       
# return only relevant information for each nearby venue
breakfast_list=[]


for v in breakfast_json:
        try:
            venue_name=v['name']
        except:
            venue_name='no data'
        try:
            venue_lat=v['location']['lat']
        except:
            venue_lat='no data'
        try:
            venue_lng=v['location']['lng']
        except:
            venue_lng='no data'
        try:
            venue_lat=v['location']['lat']
        except:
            venue_lat='no data'
        try:
            category_name=v['categories'][0]['name']
        except:
            category_name='no data'
        try:
            venue_id=v['id']
        except:
            venue_id='no data'
        
        breakfast_list.append([(
            venue_name, 
            venue_lat, 
            venue_lng, 
            category_name,
            venue_id)])

if len(breakfast_list)==0:
    print('No breakfast venues found within radius of {}m of {}'.format(radius, address))

else:
        breakfast_df = pd.DataFrame([item for venue_list in breakfast_list for item in venue_list])
        breakfast_df.columns = ['Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category','Venue ID']
        breakfast_df=breakfast_df[breakfast_df['Venue Category'].isin(breakfast_category_name_exclude)==False]
        breakfast_df.loc[:,'Venue Key']=range(len(breakfast_df.Venue))
        breakfast_df.reset_index(drop=True, inplace=True)
        print('Number of breakfast venues in {} (latitude: {},longitude: {}): {}\n'.format(address,latitude,longitude,breakfast_df.shape[0]))
        breakfast_df

KeyError: 'venues'

In [425]:
breakfast_df.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating
0,Starbucks,53.551256,10.000930,Coffee Shop,4b06afd3f964a520a3ee22e3,0,6.8
1,Enjoy Coffee Bar,53.553136,10.001385,Café,520de75b11d2a9d3ac79f1ee,1,7.6
2,Espresso House,53.551727,9.998238,Coffee Shop,5bdd8706e17910002bc67d7d,2,7.7
3,Copenhagen Coffee Lab,53.565968,9.966165,Coffee Shop,5e22e03dc8d7a3000863b375,3,6.1
4,Espresso House,53.552770,10.003656,Coffee Shop,5c6700794a7aae002cd6623a,4,5.8


#### 3.1.2 Add breakfast venues' ratings

In [426]:
# retrieve ratings of breakfast venues and append to breakfast_df (Attention: venue/rating is a premium call!)
venue_id=breakfast_df.loc[:,'Venue ID']
breakfast_df.loc[:,'Venue Rating']=0

for i in range(len(venue_id)):
#    print('\nvenue_id [',i,']:',venue_id[i])
    venue_var=venue_id[i]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_var, CLIENT_ID, CLIENT_SECRET, VERSION)
    try:
#        print('Venue rating: ', requests.get(url).json()['response']['venue']['rating'])
        breakfast_df.loc[i,'Venue Rating']=requests.get(url).json()['response']['venue']['rating']
    except:
#        print('Venue rating: This venue has not been rated yet.')
        breakfast_df.loc[i,'Venue Rating']='no rating'

breakfast_df.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating
0,Starbucks,53.551256,10.000930,Coffee Shop,4b06afd3f964a520a3ee22e3,0,no rating
1,Enjoy Coffee Bar,53.553136,10.001385,Café,520de75b11d2a9d3ac79f1ee,1,no rating
2,Espresso House,53.551727,9.998238,Coffee Shop,5bdd8706e17910002bc67d7d,2,no rating
3,Copenhagen Coffee Lab,53.565968,9.966165,Coffee Shop,5e22e03dc8d7a3000863b375,3,no rating
4,Espresso House,53.552770,10.003656,Coffee Shop,5c6700794a7aae002cd6623a,4,no rating


#### 3.1.3 Show top 6 rated breakfast venues (breakfast venues without rating dropped)

In [427]:
top_breakfast_df=breakfast_df.loc[((breakfast_df['Venue Rating']!='no rating')&(breakfast_df['Venue Category'].str.contains('estaurant')==False)&(breakfast_df['Venue Category'].str.contains('tore')==False)&(breakfast_df['Venue Category'].str.contains('akery')==False)),:].sort_values('Venue Rating', ascending=False)
top_breakfast_df.reset_index(drop=True, inplace=True)
top_breakfast_df=top_breakfast_df.head(6)
top_breakfast_df

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating


#### 3.1.4 Determine 6 breakfast areas (breakfast venues clustered based on coordinates)

3.1.4.1 Run _k_-means to cluster the breakfast venues into 6 clusters.


In [428]:
# set number of clusters
breakfast_kclusters = 6

breakfast_areas = breakfast_df.drop(['Venue','Venue Category', 'Venue ID', 'Venue Rating','Venue Key'], 1)

# run k-means clustering
breakfast_kmeans = KMeans(n_clusters=breakfast_kclusters, random_state=0).fit(breakfast_areas)

# check cluster labels generated for each row in the dataframe
breakfast_kmeans.labels_[0:10] 

array([1, 1, 1, 0, 1, 1, 0, 0, 1, 4], dtype=int32)

In [429]:
breakfast_areas.head()

,Venue Latitude,Venue Longitude
0,53.551256,10.000930
1,53.553136,10.001385
2,53.551727,9.998238
3,53.565968,9.966165
4,53.552770,10.003656


3.1.4.2 Create a dataframe that includes the 6 breakfast areas and the breakfast venues in each breakfast area.


In [430]:
# add clustering labels
try:
    breakfast_areas.insert(0, 'Item Key', breakfast_kmeans.labels_)
    breakfast_areas['Venue Key']=breakfast_df['Venue Key']
    breakfast_areas_merged = breakfast_df
except:
    breakfast_areas_merged = breakfast_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
breakfast_areas_merged = breakfast_areas_merged.join(breakfast_areas[['Venue Key','Item Key']].set_index('Venue Key'), on='Venue Key')

breakfast_areas_merged.head() # check the last columns!

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating,Item Key
0,Starbucks,53.551256,10.000930,Coffee Shop,4b06afd3f964a520a3ee22e3,0,no rating,1
1,Enjoy Coffee Bar,53.553136,10.001385,Café,520de75b11d2a9d3ac79f1ee,1,no rating,1
2,Espresso House,53.551727,9.998238,Coffee Shop,5bdd8706e17910002bc67d7d,2,no rating,1
3,Copenhagen Coffee Lab,53.565968,9.966165,Coffee Shop,5e22e03dc8d7a3000863b375,3,no rating,0
4,Espresso House,53.552770,10.003656,Coffee Shop,5c6700794a7aae002cd6623a,4,no rating,1


3.1.4.3 Determine the centroid coordinates for each breakfast area

In [431]:
# Centroid coordinates of breakfast areas
for i in breakfast_areas_merged.index:
    breakfast_areas_merged.loc[i,'Item Latitude']=breakfast_kmeans.cluster_centers_[breakfast_areas_merged['Item Key'][i]][0]
    breakfast_areas_merged.loc[i,'Item Longitude']=breakfast_kmeans.cluster_centers_[breakfast_areas_merged['Item Key'][i]][1]
breakfast_areas_merged.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating,Item Key,Item Latitude,Item Longitude
0,Starbucks,53.551256,10.000930,Coffee Shop,4b06afd3f964a520a3ee22e3,0,no rating,1,53.552170,9.993053
1,Enjoy Coffee Bar,53.553136,10.001385,Café,520de75b11d2a9d3ac79f1ee,1,no rating,1,53.552170,9.993053
2,Espresso House,53.551727,9.998238,Coffee Shop,5bdd8706e17910002bc67d7d,2,no rating,1,53.552170,9.993053
3,Copenhagen Coffee Lab,53.565968,9.966165,Coffee Shop,5e22e03dc8d7a3000863b375,3,no rating,0,53.560353,9.965718
4,Espresso House,53.552770,10.003656,Coffee Shop,5c6700794a7aae002cd6623a,4,no rating,1,53.552170,9.993053


3.1.4.4 Show the 6 breakfast areas with their geographic coordinates

In [432]:
top_breakfast_areas=breakfast_areas_merged.loc[:,['Item Key', 'Item Latitude','Item Longitude']].groupby('Item Key', as_index=False).mean()
top_breakfast_areas.loc[:,'Item Category']='Breakfast Area'
for i in top_breakfast_areas.loc[:,'Item Key']:
    top_breakfast_areas.loc[i,'Item Name']='Breakfast Area '+str(top_breakfast_areas.loc[i,'Item Key'])
top_breakfast_areas

,Item Key,Item Latitude,Item Longitude,Item Category,Item Name
0,0,53.560353,9.965718,Breakfast Area,Breakfast Area 0
1,1,53.552170,9.993053,Breakfast Area,Breakfast Area 1
2,2,53.549519,10.037344,Breakfast Area,Breakfast Area 2
3,3,53.561058,9.938500,Breakfast Area,Breakfast Area 3
4,4,53.573069,10.004401,Breakfast Area,Breakfast Area 4
5,5,53.505420,9.986052,Breakfast Area,Breakfast Area 5


#### 3.2 Search for top rated museums

##### 3.2.1 Retrieve all museums of target city as a dataframe

In [433]:
# Foursquare.com credentials and parameters
CLIENT_ID = 'R2B3BSMOAZU414BSOGLAYMB2MJPFUDT1PPEL1M0KZMD4RJFK' # your Foursquare ID
CLIENT_SECRET = '1YM0O41P05UKBSIDUTBL1G4ZYCAKEVRNQ4VFIO40THDU3EWW' # your Foursquare Secret
VERSION = '20201110' # Foursquare API version
categoryId='4bf58dd8d48988d181941735' #venue category: museum: 4bf58dd8d48988d181941735
radius=5000 # search radius around address coordinates (in m)
LIMIT = 100 # A default Foursquare API limit value

museums_category_name_exclude=[]

# create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            latitude, 
            longitude, 
            VERSION, 
            radius,
            LIMIT,
            categoryId)
            
# make the GET request
museums_json = requests.get(url).json()['response']['venues']
       
# return only relevant information for each nearby venue
museums_list=[]

for v in museums_json:
        try:
            venue_name=v['name']
        except:
            venue_name='no data'
        try:
            venue_lat=v['location']['lat']
        except:
            venue_lat='no data'
        try:
            venue_lng=v['location']['lng']
        except:
            venue_lng='no data'
        try:
            venue_lat=v['location']['lat']
        except:
            venue_lat='no data'
        try:
            category_name=v['categories'][0]['name']
        except:
            category_name='no data'
        try:
            venue_id=v['id']
        except:
            venue_id='no data'
        
        museums_list.append([(
            venue_name, 
            venue_lat, 
            venue_lng, 
            category_name,
            venue_id)])

if len(museums_list)==0:
    print('No museums found within radius of {}m of {}'.format(radius, address))

else:
        museums_df = pd.DataFrame([item for venue_list in museums_list for item in venue_list])
        museums_df.columns = ['Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category','Venue ID']
        museums_df=museums_df[museums_df['Venue Category'].isin(museums_category_name_exclude)==False]
        museums_df.loc[:,'Venue Key']=range(len(museums_df.Venue))
        museums_df.reset_index(drop=True, inplace=True)
        print('Number of museums in {} (latitude: {},longitude: {}): {}\n'.format(address,latitude,longitude,museums_df.shape[0]))
        museums_df

Number of museums in Jakarta, Indonesia (latitude: -6.1753942,longitude: 106.827183): 48



In [434]:
museums_df.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key
0,Museum Nasional Indonesia,-6.176206,106.822854,Museum,4b84944df964a520ac3c31e3,0
1,Galeri Nasional Indonesia,-6.178431,106.832302,Art Gallery,4b9ece29f964a5204e0337e3,1
2,Rimbawan Park,-6.205174,106.799119,Playground,4becc06ad6e676b0eb8b64d4,2
3,Museum Gedung Joang '45,-6.186209,106.836548,History Museum,4cf1a909c9af6dcb813e9b7f,3
4,Museum Layang-Layang Indonesia,-6.205527,106.839386,Art Museum,50d6fa10e4b09e2e5e1c62d0,4


##### 3.2.2 Add museums' ratings

In [435]:
# retrieve ratings of museums and append to museums_df (Attention: venue/rating is a premium call!)
venue_id=museums_df.loc[:,'Venue ID']
museums_df.loc[:,'Venue Rating']=0

for i in range(len(venue_id)):
#    print('\nvenue_id [',i,']:',venue_id[i])
    venue_var=venue_id[i]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_var, CLIENT_ID, CLIENT_SECRET, VERSION)
    try:
#        print('Venue rating: ', requests.get(url).json()['response']['venue']['rating'])
        museums_df.loc[i,'Venue Rating']=requests.get(url).json()['response']['venue']['rating']
    except:
#        print('Venue rating: This venue has not been rated yet.')
        museums_df.loc[i,'Venue Rating']='no rating'

museums_df.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating
0,Museum Nasional Indonesia,-6.176206,106.822854,Museum,4b84944df964a520ac3c31e3,0,8
1,Galeri Nasional Indonesia,-6.178431,106.832302,Art Gallery,4b9ece29f964a5204e0337e3,1,8
2,Rimbawan Park,-6.205174,106.799119,Playground,4becc06ad6e676b0eb8b64d4,2,no rating
3,Museum Gedung Joang '45,-6.186209,106.836548,History Museum,4cf1a909c9af6dcb813e9b7f,3,6.3
4,Museum Layang-Layang Indonesia,-6.205527,106.839386,Art Museum,50d6fa10e4b09e2e5e1c62d0,4,no rating


##### 3.2.3 Show top 6 rated museums (museums without rating dropped)

In [436]:
top_museums_df=museums_df.loc[((museums_df['Venue Rating']!='no rating')& ((museums_df['Venue Category'].str.contains('useum')))),:].sort_values('Venue Rating', ascending=False)
top_museums_df.loc[:,['Item Key','Item Name','Item Latitude','Item Longitude','Item Category']]=[top_museums_df.loc[:,'Venue Key'],top_museums_df.loc[:,'Venue'],top_museums_df.loc[:,'Venue Latitude'],top_museums_df.loc[:,'Venue Longitude'],'Top Museums']
top_museums_df.reset_index(drop=True, inplace=True)
top_museums_df=top_museums_df.head(6)
top_museums_df

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating,Item Key,Item Name,Item Latitude,Item Longitude,Item Category
0,Museum Bank Indonesia,-6.137120,106.813148,History Museum,4ba442a8f964a520e79038e3,42,8.6,42,Museum Bank Indonesia,-6.137120,106.813148,Top Museums
1,Museum Nasional Indonesia,-6.176206,106.822854,Museum,4b84944df964a520ac3c31e3,0,8,0,Museum Nasional Indonesia,-6.176206,106.822854,Top Museums
2,Museum Satria Mandala,-6.230443,106.819408,History Museum,4bf3537e706e20a1102ca898,46,7.6,46,Museum Satria Mandala,-6.230443,106.819408,Top Museums
3,Museum Sumpah Pemuda,-6.183490,106.843198,History Museum,4c7dacf618bc199c6901da58,39,7.5,39,Museum Sumpah Pemuda,-6.183490,106.843198,Top Museums
4,Museum Bank Mandiri,-6.137618,106.813232,Museum,4b9b6a34f964a520e70536e3,43,7.4,43,Museum Bank Mandiri,-6.137618,106.813232,Top Museums
5,Museum Kebangkitan Nasional,-6.178766,106.838108,History Museum,4b94b88bf964a5206d8134e3,15,7.3,15,Museum Kebangkitan Nasional,-6.178766,106.838108,Top Museums


#### 3.3 Search for top rated sights

##### 3.3.1 Retrieve all sights of target city as a dataframe

In [437]:
# Foursquare.com credentials and parameters
CLIENT_ID = 'R2B3BSMOAZU414BSOGLAYMB2MJPFUDT1PPEL1M0KZMD4RJFK' # your Foursquare ID
CLIENT_SECRET = '1YM0O41P05UKBSIDUTBL1G4ZYCAKEVRNQ4VFIO40THDU3EWW' # your Foursquare Secret
VERSION = '20201110' # Foursquare API version

categoryId='4deefb944765f83613cdba6e,5642206c498e4bfca532186c,4bf58dd8d48988d131941735,4bf58dd8d48988d126941735,56aa371be4b08b9a8d573544,4bf58dd8d48988d1e2941735,52e81612bcbc57f1066b7a22,4bf58dd8d48988d1df941735,50aaa49e4b90af0d42d5de11,56aa371be4b08b9a8d573511,56aa371be4b08b9a8d573547,4bf58dd8d48988d1e0941735,4bf58dd8d48988d15d941735,52e81612bcbc57f1066b7a14,52e81612bcbc57f1066b7a25,4bf58dd8d48988d165941735,5032848691d4c4b30a586d61,56aa371be4b08b9a8d573560,56aa371be4b08b9a8d5734c3,5bae9231bedf3950379f89c7'

#4deefb944765f83613cdba6e #Historic Site
#5642206c498e4bfca532186c #Memorial Site
#4bf58dd8d48988d131941735 #Spiritual Center (church, mosque, monastery, temple etc.)
#4bf58dd8d48988d126941735 #Government Building
#56aa371be4b08b9a8d573544 #Bay
#4bf58dd8d48988d1e2941735 #Beach
#4bf58dd8d48988d1df941735 #Bridge
#50aaa49e4b90af0d42d5de11 #Castle
#56aa371be4b08b9a8d573511 #Cave
#56aa371be4b08b9a8d573547 #Fountain
#4bf58dd8d48988d1e0941735 #Harbor / Marina
#4bf58dd8d48988d15d941735 #Lighthouse
#52e81612bcbc57f1066b7a14 #Palace
#52e81612bcbc57f1066b7a25 #Pedestrian Plaza
#4bf58dd8d48988d165941735 #Scenic Lookout
#56aa371be4b08b9a8d5734c3 #Waterfront
#5bae9231bedf3950379f89c7 #Windmill

sights_category_name_exclude=[]

radius=5000
LIMIT = 100 # A default Foursquare API limit value

# create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            latitude, 
            longitude, 
            VERSION, 
            radius,
            LIMIT,
            categoryId)
            
# make the GET request
sights_json = requests.get(url).json()['response']['venues']
       
# return only relevant information for each nearby venue
sights_list=[]


for v in sights_json:
        try:
            venue_name=v['name']
        except:
            venue_name='no data'
        try:
            venue_lat=v['location']['lat']
        except:
            venue_lat='no data'
        try:
            venue_lng=v['location']['lng']
        except:
            venue_lng='no data'
        try:
            venue_lat=v['location']['lat']
        except:
            venue_lat='no data'
        try:
            category_name=v['categories'][0]['name']
        except:
            category_name='no data'
        try:
            venue_id=v['id']
        except:
            venue_id='no data'
        
        sights_list.append([(
            venue_name, 
            venue_lat, 
            venue_lng, 
            category_name,
            venue_id)])

if len(sights_list)==0:
    print('No sights found within radius of {}m of {}'.format(radius, address))

else:
        sights_df = pd.DataFrame([item for venue_list in sights_list for item in venue_list])
        sights_df.columns = ['Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Name','Venue ID']
        sights_df=sights_df[sights_df['Venue Name'].isin(sights_category_name_exclude)==False]
        sights_df.loc[:,'Venue Key']=range(len(sights_df.Venue))
        sights_df.reset_index(drop=True, inplace=True)
        print('Number of sights in {} (latitude: {},longitude: {}): {}\n'.format(address,latitude,longitude,sights_df.shape[0]))
        sights_df

Number of sights in Jakarta, Indonesia (latitude: -6.1753942,longitude: 106.827183): 50



In [438]:
sights_df.head()

,Venue,Venue Latitude,Venue Longitude,Venue Name,Venue ID,Venue Key
0,The Art of Liu Kuo Sung,-6.175442,106.827118,Monument / Landmark,552770a7498e21e5227b160e,0
1,Monumen Nasional (MONAS),-6.175372,106.827192,Monument / Landmark,4b4d737cf964a5207dd226e3,1
2,Cek Poin Tower,-6.149008,106.849605,Lighthouse,508cd6d2498eb2451c739da2,2
3,Bundaran Hotel Indonesia (Monumen Selamat Datang),-6.195012,106.823055,Monument / Landmark,4be443639026ef3b719e0a38,3
4,Chandranaya,-6.145390,106.815550,Historic Site,5057d49ee4b0eaaf49666c93,4


##### 3.3.2 Add sights' ratings

In [439]:
# retrieve ratings of museums and append to museums_df (Attention: venue/rating is a premium call!)
venue_id=sights_df.loc[:,'Venue ID']
sights_df.loc[:,'Venue Rating']=0

for i in range(len(venue_id)):
#    print('\nvenue_id [',i,']:',venue_id[i])
    venue_var=venue_id[i]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_var, CLIENT_ID, CLIENT_SECRET, VERSION)
    try:
#        print('Venue rating: ', requests.get(url).json()['response']['venue']['rating'])
        sights_df.loc[i,'Venue Rating']=requests.get(url).json()['response']['venue']['rating']
    except:
#        print('Venue rating: This venue has not been rated yet.')
        sights_df.loc[i,'Venue Rating']='no rating'

sights_df.head()

,Venue,Venue Latitude,Venue Longitude,Venue Name,Venue ID,Venue Key,Venue Rating
0,The Art of Liu Kuo Sung,-6.175442,106.827118,Monument / Landmark,552770a7498e21e5227b160e,0,no rating
1,Monumen Nasional (MONAS),-6.175372,106.827192,Monument / Landmark,4b4d737cf964a5207dd226e3,1,7.9
2,Cek Poin Tower,-6.149008,106.849605,Lighthouse,508cd6d2498eb2451c739da2,2,no rating
3,Bundaran Hotel Indonesia (Monumen Selamat Datang),-6.195012,106.823055,Monument / Landmark,4be443639026ef3b719e0a38,3,7.8
4,Chandranaya,-6.145390,106.815550,Historic Site,5057d49ee4b0eaaf49666c93,4,no rating


##### 3.3.3 Show top 6 rated sights (sights without rating dropped)

In [440]:
#top_sights_df=sights_df.loc[((sights_df['Venue Rating']!='no rating')&(sights_df['Venue Name'].str.contains('useum'))),:].sort_values('Venue Rating', ascending=False)
top_sights_df=sights_df.loc[((sights_df['Venue Rating']!='no rating')),:].sort_values('Venue Rating', ascending=False)
top_sights_df.loc[:,['Item Key','Item Name','Item Latitude','Item Longitude','Item Category']]=[top_sights_df.loc[:,'Venue Key'],top_sights_df.loc[:,'Venue'],top_sights_df.loc[:,'Venue Latitude'],top_sights_df.loc[:,'Venue Longitude'],'Top Sights']
top_sights_df.reset_index(drop=True, inplace=True)
top_sights_df=top_sights_df.head(6)
top_sights_df

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,Venue,Venue Latitude,Venue Longitude,Venue Name,Venue ID,Venue Key,Venue Rating,Item Key,Item Name,Item Latitude,Item Longitude,Item Category
0,Patung Jenderal Sudirman,-6.203915,106.822515,Monument / Landmark,4ba48d06f964a520bba338e3,21,8.1,21,Patung Jenderal Sudirman,-6.203915,106.822515,Top Sights
1,Monumen Nasional (MONAS),-6.175372,106.827192,Monument / Landmark,4b4d737cf964a5207dd226e3,1,7.9,1,Monumen Nasional (MONAS),-6.175372,106.827192,Top Sights
2,Bundaran Hotel Indonesia (Monumen Selamat Datang),-6.195012,106.823055,Monument / Landmark,4be443639026ef3b719e0a38,3,7.8,3,Bundaran Hotel Indonesia (Monumen Selamat Datang),-6.195012,106.823055,Top Sights
3,Bank Indonesia,-6.181942,106.821871,Government Building,4b99dc7ef964a520da9435e3,11,7.8,11,Bank Indonesia,-6.181942,106.821871,Top Sights
4,Kota Tua,-6.135388,106.812949,Historic Site,4d330bf1c6cba35d2c12277a,23,7.8,23,Kota Tua,-6.135388,106.812949,Top Sights
5,Kementerian Negara Koperasi dan UKM,-6.219321,106.831273,Government Building,4be3d85863609c7406f61bff,29,6.2,29,Kementerian Negara Koperasi dan UKM,-6.219321,106.831273,Top Sights


#### 3.4 Search for areas with many restaurants

##### 3.4.1 Retrieve all restaurants of target city as a dataframe

In [441]:
# Foursquare.com credentials and parameters
CLIENT_ID = 'R2B3BSMOAZU414BSOGLAYMB2MJPFUDT1PPEL1M0KZMD4RJFK' # your Foursquare ID
CLIENT_SECRET = '1YM0O41P05UKBSIDUTBL1G4ZYCAKEVRNQ4VFIO40THDU3EWW' # your Foursquare Secret
VERSION = '20201110' # Foursquare API version

categoryId='4d4b7105d754a06374d81259'

#Food
#4d4b7105d754a06374d81259

restaurants_category_exclude=['Bagel Shop','Bakery','Breakfast Spot','Bubble Tea Shop','Cafeteria','Café','Candy Store','Coffee Shop','Deli / Bodega','Dessert Shop','Diner','Donut Shop','Fast Food Restaurant','Fish & Chips Shop','Food & Drink Shop',
                       'Food Stand','Food Truck','Fried Chicken Joint','Ice Cream Shop','Hotel','Rest Area',
                       'Friterie','Gastropub','Hot Dog Joint','Irish Pub','Juice Bar','Pet Café','Pizza Place','Snack Place','Soup Place','Tea Room','Wings Joint','Bagel Shop','Bakery','Breakfast Spot','Bubble Tea Shop','Cafeteria',
                       'Café','Coffee Shop','Deli / Bodega','Dessert Shop','Diner','Donut Shop','Fast Food Restaurant','Fish & Chips Shop','Food Stand','Food Truck','Fried Chicken Joint','Friterie','Gastropub','Hot Dog Joint',
                       'Irish Pub','Juice Bar','Pet Café','Pizza Place','Snack Place','Soup Place','Tea Room','Wings Joint','Market','Pastry Shop']

radius=5000
LIMIT = 100 # A default Foursquare API limit value

# create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            latitude, 
            longitude, 
            VERSION, 
            radius,
            LIMIT,
            categoryId)
            
# make the GET request
restaurants_json = requests.get(url).json()['response']['venues']
       
# return only relevant information for each nearby venue
restaurants_list=[]


for v in restaurants_json:
        try:
            venue_name=v['name']
        except:
            venue_name='no data'
        try:
            venue_lat=v['location']['lat']
        except:
            venue_lat='no data'
        try:
            venue_lng=v['location']['lng']
        except:
            venue_lng='no data'
        try:
            venue_lat=v['location']['lat']
        except:
            venue_lat='no data'
        try:
            category_name=v['categories'][0]['name']
        except:
            category_name='no data'
        try:
            venue_id=v['id']
        except:
            venue_id='no data'
        
        restaurants_list.append([(
            venue_name, 
            venue_lat, 
            venue_lng, 
            category_name,
            venue_id)])

if len(breakfast_list)==0:
    print('No restaurants found within radius of {}m of {}'.format(radius, address))

else:
        restaurants_df = pd.DataFrame([item for venue_list in restaurants_list for item in venue_list])
        restaurants_df.columns = ['Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category','Venue ID']
        restaurants_df=restaurants_df[restaurants_df['Venue Category'].isin(restaurants_category_exclude)==False]
        restaurants_df.loc[:,'Venue Key']=range(len(restaurants_df.Venue))
        restaurants_df.reset_index(drop=True, inplace=True)
        print('Number of restaurants in {} (latitude: {},longitude: {}): {}\n'.format(address,latitude,longitude,restaurants_df.shape[0]))
        restaurants_df

Number of restaurants in Jakarta, Indonesia (latitude: -6.1753942,longitude: 106.827183): 27



In [442]:
restaurants_df.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key
0,Joni Steak,-6.161584,106.831039,Steakhouse,4c5424f9479fc9280511a192,0
1,Malacca Toast Sunter,-6.139500,106.862230,Malay Restaurant,5bdda6e381635b002c5ecde0,1
2,Bakso Solo Samrat,-6.149853,106.817612,Indonesian Meatball Place,5784ea1d38fa06b8a3bd9b22,2
3,starbucks,-6.146693,106.816060,Southern / Soul Food Restaurant,5072e388e4b01785bacd9652,3
4,Asli Sate Domba Afrika,-6.217994,106.819519,Mediterranean Restaurant,4cc3cb78be40a35da8de7a4c,4


#### 3.4.3 Add restaurants' ratings

In [443]:
# retrieve ratings of restaurants and append to restaurants_df (Attention: venue/rating is a premium call!)
venue_id=restaurants_df.loc[:,'Venue ID']
restaurants_df.loc[:,'Venue Rating']=0

for i in range(len(venue_id)):
#    print('\nvenue_id [',i,']:',venue_id[i])
    venue_var=venue_id[i]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_var, CLIENT_ID, CLIENT_SECRET, VERSION)
    try:
#        print('Venue rating: ', requests.get(url).json()['response']['venue']['rating'])
        restaurants_df.loc[i,'Venue Rating']=requests.get(url).json()['response']['venue']['rating']
    except:
#        print('Venue rating: This venue has not been rated yet.')
        restaurants_df.loc[i,'Venue Rating']='no rating'

restaurants_df.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating
0,Joni Steak,-6.161584,106.831039,Steakhouse,4c5424f9479fc9280511a192,0,7.6
1,Malacca Toast Sunter,-6.139500,106.862230,Malay Restaurant,5bdda6e381635b002c5ecde0,1,no rating
2,Bakso Solo Samrat,-6.149853,106.817612,Indonesian Meatball Place,5784ea1d38fa06b8a3bd9b22,2,no rating
3,starbucks,-6.146693,106.816060,Southern / Soul Food Restaurant,5072e388e4b01785bacd9652,3,no rating
4,Asli Sate Domba Afrika,-6.217994,106.819519,Mediterranean Restaurant,4cc3cb78be40a35da8de7a4c,4,no rating


#### 3.4.4 Show top 100 rated restaurants (restaurants without rating dropped)

In [444]:
#top_restaurants_df=restaurants_df.loc[((restaurants_df['Venue Rating']!='no rating')&(restaurants_df['Venue Name'].str.contains('estaurant')==False)&(restaurants_df['Venue Name'].str.contains('tore')==False)&(restaurants_df['Venue Name'].str.contains('akery')==False)),:].sort_values('Venue Rating', ascending=False)
top_restaurants_df=restaurants_df.loc[((restaurants_df['Venue Rating']!='no rating')),:].sort_values('Venue Rating', ascending=False)
top_restaurants_df.reset_index(drop=True, inplace=True)
top_restaurants_df=top_restaurants_df.head(100)
top_restaurants_df

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating
0,SKYE,-6.196795,106.822580,Restaurant,4fa12391e4b0c33ec67cd84c,15,8.9
1,Paulaner Bräuhaus,-6.195836,106.822884,German Restaurant,4c09e1e9009a0f470aa0e8bf,13,8.8
2,Bluegrass Bar & Grill,-6.219456,106.834365,Bar,4e018f8d1f6ef39c29439776,17,8.6
3,Sun City Restaurant and Luxury Club,-6.145806,106.816925,Dim Sum Restaurant,4d5118c09b27721e1722b446,24,8.4
4,Bakmi Sui Sen,-6.163371,106.836239,Noodle House,4c1708bbdaf42d7fdb264566,22,8.2
5,Gado Gado Tamansari Raya,-6.152726,106.827043,Salad Place,4c38bbac6ec69c749dc704a9,25,8.1
6,Shantung Restaurant,-6.163381,106.836240,Chinese Restaurant,5059bc77e4b041be10b3e675,20,8
7,Bakmi Aboen,-6.162442,106.833490,Noodle House,4da9241af7b145b36ff0b50e,14,7.9
8,FOODPRINT,-6.195241,106.820244,Food Court,59ede968c4df1d5067bc5291,16,7.9
9,A&W Golden Truly,-6.163381,106.836243,American Restaurant,4f617e70e4b0dd26f19a1f95,19,7.9


#### 3.4.5 Determine 6 dining areas (restaurants clustered into 6 clusters based on coordinates)

3.4.5.1 Run _k_-means to cluster the restaurants into 6 clusters.


In [445]:
# set number of clusters
restaurants_kclusters = 6

dining_areas = restaurants_df.drop(['Venue','Venue Category', 'Venue ID', 'Venue Rating','Venue Key'], 1)

# run k-means clustering
restaurants_kmeans = KMeans(n_clusters=restaurants_kclusters, random_state=0).fit(dining_areas)

# check cluster labels generated for each row in the dataframe
restaurants_kmeans.labels_[0:10] 

array([2, 3, 0, 0, 1, 2, 0, 1, 3, 5], dtype=int32)

In [446]:
dining_areas.head()

,Venue Latitude,Venue Longitude
0,-6.161584,106.831039
1,-6.139500,106.862230
2,-6.149853,106.817612
3,-6.146693,106.816060
4,-6.217994,106.819519


3.4.5.2 Create a dataframe that includes the dining areas and the restaurants in each dining area.


In [447]:
# add clustering labels
try:
    dining_areas.insert(0, 'Item Key', restaurants_kmeans.labels_)
    dining_areas['Venue Key']=restaurants_df['Venue Key']
    dining_areas_merged = restaurants_df
except:
    dining_areas_merged = restaurants_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
dining_areas_merged = dining_areas_merged.join(dining_areas[['Venue Key','Item Key']].set_index('Venue Key'), on='Venue Key')

dining_areas_merged.head() # check the last columns!

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating,Item Key
0,Joni Steak,-6.161584,106.831039,Steakhouse,4c5424f9479fc9280511a192,0,7.6,2
1,Malacca Toast Sunter,-6.139500,106.862230,Malay Restaurant,5bdda6e381635b002c5ecde0,1,no rating,3
2,Bakso Solo Samrat,-6.149853,106.817612,Indonesian Meatball Place,5784ea1d38fa06b8a3bd9b22,2,no rating,0
3,starbucks,-6.146693,106.816060,Southern / Soul Food Restaurant,5072e388e4b01785bacd9652,3,no rating,0
4,Asli Sate Domba Afrika,-6.217994,106.819519,Mediterranean Restaurant,4cc3cb78be40a35da8de7a4c,4,no rating,1


3.4.5.3 Determine centroid coordinates of dining areas

In [448]:
# Centroid coordinates
for i in dining_areas_merged.index:
    dining_areas_merged.loc[i,'Item Latitude']=restaurants_kmeans.cluster_centers_[dining_areas_merged['Item Key'][i]][0]
    dining_areas_merged.loc[i,'Item Longitude']=restaurants_kmeans.cluster_centers_[dining_areas_merged['Item Key'][i]][1]
dining_areas_merged.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating,Item Key,Item Latitude,Item Longitude
0,Joni Steak,-6.161584,106.831039,Steakhouse,4c5424f9479fc9280511a192,0,7.6,2,-6.159582,106.835583
1,Malacca Toast Sunter,-6.139500,106.862230,Malay Restaurant,5bdda6e381635b002c5ecde0,1,no rating,3,-6.144373,106.862813
2,Bakso Solo Samrat,-6.149853,106.817612,Indonesian Meatball Place,5784ea1d38fa06b8a3bd9b22,2,no rating,0,-6.154077,106.813941
3,starbucks,-6.146693,106.816060,Southern / Soul Food Restaurant,5072e388e4b01785bacd9652,3,no rating,0,-6.154077,106.813941
4,Asli Sate Domba Afrika,-6.217994,106.819519,Mediterranean Restaurant,4cc3cb78be40a35da8de7a4c,4,no rating,1,-6.216916,106.825365


3.4.5.4 Show the 6 dining areas with their coordinates

In [449]:
top_dining_areas=dining_areas_merged.loc[:,['Item Key', 'Item Latitude','Item Longitude']].groupby('Item Key', as_index=False).mean()
top_dining_areas.loc[:,'Item Category']='Dining Area'
for i in top_dining_areas.loc[:,'Item Key']:
    top_dining_areas.loc[i,'Item Name']='Dining Area '+str(top_dining_areas.loc[i,'Item Key'])
top_dining_areas

,Item Key,Item Latitude,Item Longitude,Item Category,Item Name
0,0,-6.154077,106.813941,Dining Area,Dining Area 0
1,1,-6.216916,106.825365,Dining Area,Dining Area 1
2,2,-6.159582,106.835583,Dining Area,Dining Area 2
3,3,-6.144373,106.862813,Dining Area,Dining Area 3
4,4,-6.195957,106.821903,Dining Area,Dining Area 4
5,5,-6.177534,106.790108,Dining Area,Dining Area 5


#### 3.5 Search for areas with top rated nightlife venues

##### 3.5.1 Retrieve all bars of target city as a dataframe

In [450]:
# Foursquare.com credentials and parameters
CLIENT_ID = 'R2B3BSMOAZU414BSOGLAYMB2MJPFUDT1PPEL1M0KZMD4RJFK' # your Foursquare ID
CLIENT_SECRET = '1YM0O41P05UKBSIDUTBL1G4ZYCAKEVRNQ4VFIO40THDU3EWW' # your Foursquare Secret
VERSION = '20201110' # Foursquare API version

categoryId='4d4b7105d754a06376d81259,52e81612bcbc57f1066b79ec,52e81612bcbc57f1066b7a06'

#Nightlife Spot
#4d4b7105d754a06376d81259
#Salsa Club
#52e81612bcbc57f1066b79ec
#Irish Pub
#52e81612bcbc57f1066b7a06

bars_category_name_exclude=[]

radius=5000
LIMIT = 100 # A default Foursquare API limit value

# create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            latitude, 
            longitude, 
            VERSION, 
            radius,
            LIMIT,
            categoryId)
            
# make the GET request
bars_json = requests.get(url).json()['response']['venues']
       
# return only relevant information for each nearby venue
bars_list=[]


for v in bars_json:
        try:
            venue_name=v['name']
        except:
            venue_name='no data'
        try:
            venue_lat=v['location']['lat']
        except:
            venue_lat='no data'
        try:
            venue_lng=v['location']['lng']
        except:
            venue_lng='no data'
        try:
            venue_lat=v['location']['lat']
        except:
            venue_lat='no data'
        try:
            category_name=v['categories'][0]['name']
        except:
            category_name='no data'
        try:
            venue_id=v['id']
        except:
            venue_id='no data'
        
        bars_list.append([(
            venue_name, 
            venue_lat, 
            venue_lng, 
            category_name,
            venue_id)])

if len(bars_list)==0:
    print('No bars found within radius of {}m of {}'.format(radius, address))

else:
        bars_df = pd.DataFrame([item for venue_list in bars_list for item in venue_list])
        bars_df.columns = ['Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category','Venue ID']
        bars_df=bars_df[bars_df['Venue Category'].isin(bars_category_name_exclude)==False]
        bars_df.loc[:,'Venue Key']=range(len(bars_df.Venue))
        bars_df.reset_index(drop=True, inplace=True)
        print('Number of bars in {} (latitude: {},longitude: {}): {}\n'.format(address,latitude,longitude,bars_df.shape[0]))
        bars_df

Number of bars in Jakarta, Indonesia (latitude: -6.1753942,longitude: 106.827183): 50



In [451]:
bars_df.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key
0,MORRISSEY,-6.186951,106.829162,Hotel,4e7bf76a483bfd57d6af8ce3,0
1,HapPup! KTV,-6.224550,106.828990,Karaoke Bar,590c7575ad1ea454c4e0ed84,1
2,Sanjiken,-6.226426,106.807211,Sake Bar,53f21c4d498ee2a0d41c8b01,2
3,My Brew,-6.146334,106.863478,Brewery,4d598425f8ff236a0f45cc4a,3
4,Boca Rica,-6.221503,106.814499,Lounge,5c82b03a2f97ec002cb04609,4


##### 3.5.2 Add bars' ratings

In [452]:
# retrieve ratings of bars and append to museums_df (Attention: venue/rating is a premium call!)
venue_id=bars_df.loc[:,'Venue ID']
bars_df.loc[:,'Venue Rating']=0

for i in range(len(venue_id)):
#    print('\nvenue_id [',i,']:',venue_id[i])
    venue_var=venue_id[i]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_var, CLIENT_ID, CLIENT_SECRET, VERSION)
    try:
#        print('Venue rating: ', requests.get(url).json()['response']['venue']['rating'])
        bars_df.loc[i,'Venue Rating']=requests.get(url).json()['response']['venue']['rating']
    except:
#        print('Venue rating: This venue has not been rated yet.')
        bars_df.loc[i,'Venue Rating']='no rating'

bars_df.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating
0,MORRISSEY,-6.186951,106.829162,Hotel,4e7bf76a483bfd57d6af8ce3,0,8.1
1,HapPup! KTV,-6.224550,106.828990,Karaoke Bar,590c7575ad1ea454c4e0ed84,1,no rating
2,Sanjiken,-6.226426,106.807211,Sake Bar,53f21c4d498ee2a0d41c8b01,2,no rating
3,My Brew,-6.146334,106.863478,Brewery,4d598425f8ff236a0f45cc4a,3,no rating
4,Boca Rica,-6.221503,106.814499,Lounge,5c82b03a2f97ec002cb04609,4,no rating


##### 3.5.3 Show top 5 rated bars (bars without rating dropped)

In [453]:
#top_bars_df=bars_df.loc[((bars_df['Venue Rating']!='no rating')&(bars_df['Venue Name'].str.contains('estaurant')==False)),:].sort_values('Venue Rating', ascending=False)
top_bars_df=bars_df.loc[((bars_df['Venue Rating']!='no rating')),:].sort_values('Venue Rating', ascending=False)
top_bars_df.reset_index(drop=True, inplace=True)
top_bars_df=top_bars_df.head(5)
top_bars_df

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating
0,Cork&Screw,-6.225700,106.808919,Wine Bar,54929b71498ecb80c59c7382,34,9
1,Colosseum Club,-6.132878,106.813873,Nightclub,52752cce498e88a71abd7f82,48,8.9
2,SKYE,-6.196795,106.822580,Restaurant,4fa12391e4b0c33ec67cd84c,23,8.9
3,Hard Rock Cafe Jakarta,-6.225280,106.810270,American Restaurant,51f34889498e9ce99e5a16fa,49,8.8
4,Paulaner Bräuhaus,-6.195836,106.822884,German Restaurant,4c09e1e9009a0f470aa0e8bf,19,8.8


##### 3.5.4 Determine 6 nightlife areas (bars clustered into 6 clusters based on coordinates)

3.5.4.1 Run _k_-means to cluster the bars into 6 clusters.


In [454]:
# set number of clusters
bars_kclusters = 6

bars_areas = bars_df.drop(['Venue','Venue Category', 'Venue ID', 'Venue Rating','Venue Key'], 1)

# run k-means clustering
bars_kmeans = KMeans(n_clusters=bars_kclusters, random_state=0).fit(bars_areas)

# check cluster labels generated for each row in the dataframe
bars_kmeans.labels_[0:10] 

array([2, 3, 0, 5, 0, 3, 0, 1, 3, 0], dtype=int32)

In [455]:
bars_areas.head()

,Venue Latitude,Venue Longitude
0,-6.186951,106.829162
1,-6.224550,106.828990
2,-6.226426,106.807211
3,-6.146334,106.863478
4,-6.221503,106.814499


##### 3.5.4.2 Create a dataframe that shows the 6 nightlife areas and the bars in each nightlife area.


In [456]:
# add clustering labels
try:
    bars_areas.insert(0, 'Item Key', bars_kmeans.labels_)
    bars_areas['Venue Key']=bars_df['Venue Key']
    bars_areas_merged = bars_df
except:
    bars_areas_merged = bars_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
bars_areas_merged = bars_areas_merged.join(bars_areas[['Venue Key','Item Key']].set_index('Venue Key'), on='Venue Key')

bars_areas_merged.head() # check the last columns!

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating,Item Key
0,MORRISSEY,-6.186951,106.829162,Hotel,4e7bf76a483bfd57d6af8ce3,0,8.1,2
1,HapPup! KTV,-6.224550,106.828990,Karaoke Bar,590c7575ad1ea454c4e0ed84,1,no rating,3
2,Sanjiken,-6.226426,106.807211,Sake Bar,53f21c4d498ee2a0d41c8b01,2,no rating,0
3,My Brew,-6.146334,106.863478,Brewery,4d598425f8ff236a0f45cc4a,3,no rating,5
4,Boca Rica,-6.221503,106.814499,Lounge,5c82b03a2f97ec002cb04609,4,no rating,0


3.5.4.3 Determine centroid coordinates of nightlife areas

In [457]:
# Centroid coordinates of nightlife areas
for i in bars_areas_merged.index:
    bars_areas_merged.loc[i,'Item Latitude']=bars_kmeans.cluster_centers_[bars_areas_merged['Item Key'][i]][0]
    bars_areas_merged.loc[i,'Item Longitude']=bars_kmeans.cluster_centers_[bars_areas_merged['Item Key'][i]][1]
bars_areas_merged.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating,Item Key,Item Latitude,Item Longitude
0,MORRISSEY,-6.186951,106.829162,Hotel,4e7bf76a483bfd57d6af8ce3,0,8.1,2,-6.198135,106.826734
1,HapPup! KTV,-6.224550,106.828990,Karaoke Bar,590c7575ad1ea454c4e0ed84,1,no rating,3,-6.222659,106.833535
2,Sanjiken,-6.226426,106.807211,Sake Bar,53f21c4d498ee2a0d41c8b01,2,no rating,0,-6.222949,106.810011
3,My Brew,-6.146334,106.863478,Brewery,4d598425f8ff236a0f45cc4a,3,no rating,5,-6.154381,106.849596
4,Boca Rica,-6.221503,106.814499,Lounge,5c82b03a2f97ec002cb04609,4,no rating,0,-6.222949,106.810011


3.5.4.3 Show nightlife areas and their coordinates

In [458]:
top_nightlife_areas=bars_areas_merged.loc[:,['Item Key', 'Item Latitude','Item Longitude']].groupby('Item Key', as_index=False).mean()
top_nightlife_areas.loc[:,'Item Category']='Nightlife Area'
for i in top_nightlife_areas.loc[:,'Item Key']:
    top_nightlife_areas.loc[i,'Item Name']='Nightlife Area '+str(top_nightlife_areas.loc[i,'Item Key'])
top_nightlife_areas

,Item Key,Item Latitude,Item Longitude,Item Category,Item Name
0,0,-6.222949,106.810011,Nightlife Area,Nightlife Area 0
1,1,-6.147751,106.814530,Nightlife Area,Nightlife Area 1
2,2,-6.198135,106.826734,Nightlife Area,Nightlife Area 2
3,3,-6.222659,106.833535,Nightlife Area,Nightlife Area 3
4,4,-6.180470,106.792451,Nightlife Area,Nightlife Area 4
5,5,-6.154381,106.849596,Nightlife Area,Nightlife Area 5


#### 3.6 Determine 6 unique day-long itineraries

3.6.1 Generate dataframe with coordinates of top_breakfast_areas, top_museums, top_sights, top_dining_ares, top_nightlife_areas

In [459]:
day_df=pd.DataFrame()
day_df=top_breakfast_areas
day_df=day_df.append(top_museums_df[['Item Key', 'Item Name','Item Latitude', 'Item Longitude','Item Category']])
day_df=day_df.append(top_sights_df[['Item Key', 'Item Latitude', 'Item Longitude','Item Name','Item Category']])
day_df=day_df.append(top_dining_areas[['Item Key', 'Item Latitude', 'Item Longitude','Item Name','Item Category']])
day_df=day_df.append(top_nightlife_areas[['Item Key', 'Item Latitude', 'Item Longitude','Item Name','Item Category']])
day_df=day_df[['Item Category', 'Item Name','Item Key', 'Item Latitude', 'Item Longitude']]
day_df.reset_index(drop=True, inplace=True)
day_df.loc[:,'Item Key']=day_df.index
day_df

,Item Category,Item Name,Item Key,Item Latitude,Item Longitude
0,Breakfast Area,Breakfast Area 0,0,53.560353,9.965718
1,Breakfast Area,Breakfast Area 1,1,53.552170,9.993053
2,Breakfast Area,Breakfast Area 2,2,53.549519,10.037344
3,Breakfast Area,Breakfast Area 3,3,53.561058,9.938500
4,Breakfast Area,Breakfast Area 4,4,53.573069,10.004401
5,Breakfast Area,Breakfast Area 5,5,53.505420,9.986052
6,Top Museums,Museum Bank Indonesia,6,-6.137120,106.813148
7,Top Museums,Museum Nasional Indonesia,7,-6.176206,106.822854
8,Top Museums,Museum Satria Mandala,8,-6.230443,106.819408
9,Top Museums,Museum Sumpah Pemuda,9,-6.183490,106.843198


3.6.2 Run _k_-means to cluster the breakfast venues into 6 clusters.


In [460]:
# set number of clusters
day_kclusters = 6

day_areas = day_df.drop(['Item Category','Item Name'], 1)

# run k-means clustering
day_kmeans = KMeans(n_clusters=day_kclusters, random_state=0).fit(day_areas)

# check cluster labels generated for each row in the dataframe
day_kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 3, 3, 3, 3], dtype=int32)

In [461]:
day_areas.head()

,Item Key,Item Latitude,Item Longitude
0,0,53.560353,9.965718
1,1,53.552170,9.993053
2,2,53.549519,10.037344
3,3,53.561058,9.938500
4,4,53.573069,10.004401


3.6.3 Create a dataframe that includes the itineraries and the items in each itinerary.


In [462]:
# add clustering labels
try:
    day_areas.insert(0, 'Day Area', day_kmeans.labels_)
#    day_areas['Venue Key']=day_df['Venue Key']
    day_areas_merged = day_df
except:
    day_areas_merged = day_df

# merge day_areas_merged with day_areas to add latitude/longitude for each area
day_areas_merged = day_areas_merged.join(day_areas[['Item Key','Day Area']].set_index('Item Key'), on='Item Key')

day_areas_merged.head() # check the last columns!

,Item Category,Item Name,Item Key,Item Latitude,Item Longitude,Day Area
0,Breakfast Area,Breakfast Area 0,0,53.560353,9.965718,1
1,Breakfast Area,Breakfast Area 1,1,53.552170,9.993053,1
2,Breakfast Area,Breakfast Area 2,2,53.549519,10.037344,1
3,Breakfast Area,Breakfast Area 3,3,53.561058,9.938500,1
4,Breakfast Area,Breakfast Area 4,4,53.573069,10.004401,1


In [463]:
# Centroid coordinates of breakfast areas
for i in day_areas_merged.index:
    day_areas_merged.loc[i,'Day Area Latitude']=day_kmeans.cluster_centers_[day_areas_merged['Day Area'][i]][1]
    day_areas_merged.loc[i,'Day Area Longitude']=day_kmeans.cluster_centers_[day_areas_merged['Day Area'][i]][2]
day_areas_merged.head()

,Item Category,Item Name,Item Key,Item Latitude,Item Longitude,Day Area,Day Area Latitude,Day Area Longitude
0,Breakfast Area,Breakfast Area 0,0,53.560353,9.965718,1,53.550265,9.987511
1,Breakfast Area,Breakfast Area 1,1,53.552170,9.993053,1,53.550265,9.987511
2,Breakfast Area,Breakfast Area 2,2,53.549519,10.037344,1,53.550265,9.987511
3,Breakfast Area,Breakfast Area 3,3,53.561058,9.938500,1,53.550265,9.987511
4,Breakfast Area,Breakfast Area 4,4,53.573069,10.004401,1,53.550265,9.987511


3.6.4 Calculate distances between breakfast areas and top museums

In [464]:
import math
d1=pd.DataFrame()
for i in range(6):  # loop through breakfast areas
    for j in range(6): # loop through top museums
        d1.loc[i,j]=math.sqrt( (top_museums_df.loc[j,'Item Latitude']-top_breakfast_areas.loc[i,'Item Latitude'])**2 + (top_museums_df.loc[j,'Item Longitude']-top_breakfast_areas.loc[i,'Item Longitude'])**2)
d1=d1.set_index(top_breakfast_areas.loc[:,'Item Name'].unique())
d1.columns=top_museums_df.loc[:,'Item Name'].unique()
d1    

,Museum Bank Indonesia,Museum Nasional Indonesia,Museum Satria Mandala,Museum Sumpah Pemuda,Museum Bank Mandiri,Museum Kebangkitan Nasional
Breakfast Area 0,113.768243,113.797018,113.822566,113.818158,113.768576,113.811346
Breakfast Area 1,113.740679,113.769456,113.795008,113.790595,113.741012,113.783783
Breakfast Area 2,113.701588,113.730370,113.755931,113.751509,113.701921,113.744696
Breakfast Area 3,113.791783,113.820554,113.846097,113.841695,113.792115,113.834883
Breakfast Area 4,113.741989,113.770772,113.796333,113.791910,113.742322,113.785098
Breakfast Area 5,113.722114,113.750880,113.776413,113.772021,113.722446,113.765209


3.6.5 Calculate distances between top museums and top sights

In [465]:
import math
d2=pd.DataFrame()
for i in range(6):  # loop through top museums
    for j in range(6): # loop through top sights
        d2.loc[i,j]=math.sqrt( (top_sights_df.loc[j,'Item Latitude']-top_museums_df.loc[i,'Item Latitude'])**2 + (top_sights_df.loc[j,'Item Longitude']-top_museums_df.loc[i,'Item Longitude'])**2)
d2=d2.set_index(top_museums_df.loc[:,'Item Name'].unique())
d2.columns=top_sights_df.loc[:,'Item Name'].unique()
d2    

,Patung Jenderal Sudirman,Monumen Nasional (MONAS),Bundaran Hotel Indonesia (Monumen Selamat Datang),Bank Indonesia,Kota Tua,Kementerian Negara Koperasi dan UKM
Museum Bank Indonesia,0.067448,0.040748,0.058734,0.045663,0.001743,0.084176
Museum Nasional Indonesia,0.027711,0.004418,0.018808,0.005820,0.042002,0.043930
Museum Satria Mandala,0.026710,0.055619,0.035618,0.048564,0.095274,0.016263
Museum Sumpah Pemuda,0.029068,0.017947,0.023206,0.021383,0.056823,0.037763
Museum Bank Mandiri,0.066943,0.040252,0.058228,0.045158,0.002248,0.083671
Museum Kebangkitan Nasional,0.029591,0.011431,0.022148,0.016545,0.050146,0.041127


3.6.6 Calculate distances between top sights and nearest dining area

In [466]:
import math
d3=pd.DataFrame()
for i in range(6):  # loop through top museums
    for j in range(6): # loop through top sights
        d3.loc[i,j]=math.sqrt( (top_dining_areas.loc[j,'Item Latitude']-top_sights_df.loc[i,'Item Latitude'])**2 + (top_dining_areas.loc[j,'Item Longitude']-top_sights_df.loc[i,'Item Longitude'])**2)
d3=d3.set_index(top_sights_df.loc[:,'Item Name'].unique())
d3.columns=top_dining_areas.loc[:,'Item Name'].unique()
d3    

,Dining Area 0,Dining Area 1,Dining Area 2,Dining Area 3,Dining Area 4,Dining Area 5
Patung Jenderal Sudirman,0.050570,0.013310,0.046218,0.071897,0.007981,0.041787
Monumen Nasional (MONAS),0.025081,0.041584,0.017880,0.047221,0.021254,0.037147
Bundaran Hotel Indonesia (Monumen Selamat Datang),0.041938,0.022025,0.037580,0.064383,0.001490,0.037296
Bank Indonesia,0.028972,0.035147,0.026230,0.055568,0.014015,0.032067
Kota Tua,0.018715,0.082467,0.033131,0.050667,0.061227,0.047937
Kementerian Negara Koperasi dan UKM,0.067507,0.006380,0.059894,0.081314,0.025173,0.058658


3.6.7 Calculate distances between dining area and nearest nightlife area

In [467]:
import math
d4=pd.DataFrame()
for i in range(6):  # loop through top museums
    for j in range(6): # loop through top sights
        d4.loc[i,j]=math.sqrt( (top_nightlife_areas.loc[j,'Item Latitude']-top_dining_areas.loc[i,'Item Latitude'])**2 + (top_nightlife_areas.loc[j,'Item Longitude']-top_dining_areas.loc[i,'Item Longitude'])**2)
d4=d4.set_index(top_dining_areas.loc[:,'Item Name'].unique())
d4.columns=top_nightlife_areas.loc[:,'Item Name'].unique()
d4    

,Nightlife Area 0,Nightlife Area 1,Nightlife Area 2,Nightlife Area 3,Nightlife Area 4,Nightlife Area 5
Dining Area 0,0.068984,0.006353,0.045878,0.071327,0.034036,0.035657
Dining Area 1,0.016496,0.070008,0.018830,0.009987,0.049108,0.067065
Dining Area 2,0.068331,0.024150,0.039556,0.063110,0.047924,0.014947
Dining Area 3,0.094669,0.048402,0.064747,0.083583,0.079082,0.016579
Dining Area 4,0.029494,0.048767,0.005299,0.029126,0.033275,0.049955
Dining Area 5,0.049585,0.038516,0.042022,0.062627,0.003757,0.063835


3.6.8 Generate dataframe with optimum itineraries

In [468]:
day_trips_df=pd.DataFrame()
for i in range(6):
    day_trips_df.loc[i,'Breakfast Area']=d1.loc[d1.iloc[:,i]==d1.iloc[:,i].min(),d1.columns[i]].index[0]
    day_trips_df.loc[i,'Museum']=d1.columns[i]
    day_trips_df.loc[i,'Sight']=d2.T.loc[d2.T.iloc[:,i]==d2.T.iloc[:,i].min(),d2.T.columns[i]].index[0]
    day_trips_df.loc[i,'Dining Area']=d3.T.loc[d3.T.loc[:,d3.T.loc[:,d3.T.columns==day_trips_df.loc[i,'Sight']].min().index[0]]==d3.T.loc[:,d3.T.loc[:,d3.T.columns==day_trips_df.loc[i,'Sight']].min().index[0]].min(),d3.T.loc[:,d3.T.columns==day_trips_df.loc[i,'Sight']].min().index[0]].index[0]
    day_trips_df.loc[i,'Nightlife Area']=d4.T.loc[d4.T.loc[:,d4.T.loc[:,d4.T.columns==day_trips_df.loc[i,'Dining Area']].min().index[0]]==d4.T.loc[:,d4.T.loc[:,d4.T.columns==day_trips_df.loc[i,'Dining Area']].min().index[0]].min(),d4.T.loc[:,d4.T.columns==day_trips_df.loc[i,'Dining Area']].min().index[0]].index[0]
    if i<6:
        d1=d1.drop(d1.loc[d1.iloc[:,i]==d1.iloc[:,i].min(),d1.columns[i]].index[0],axis=0)
        d2=d2.drop(d2.T.loc[d2.T.iloc[:,i]==d2.T.iloc[:,i].min(),d2.T.columns[i]].index[0],axis=1)
        d3=d3.drop(d3.T.loc[d3.T.loc[:,d3.T.loc[:,d3.T.columns==day_trips_df.loc[i,'Sight']].min().index[0]]==d3.T.loc[:,d3.T.loc[:,d3.T.columns==day_trips_df.loc[i,'Sight']].min().index[0]].min(),d3.T.loc[:,d3.T.columns==day_trips_df.loc[i,'Sight']].min().index[0]].index[0],axis=1)
        d4=d4.drop(d4.T.loc[d4.T.loc[:,d4.T.loc[:,d4.T.columns==day_trips_df.loc[i,'Dining Area']].min().index[0]]==d4.T.loc[:,d4.T.loc[:,d4.T.columns==day_trips_df.loc[i,'Dining Area']].min().index[0]].min(),d4.T.loc[:,d4.T.columns==day_trips_df.loc[i,'Dining Area']].min().index[0]].index[0] ,axis=1)
    else:
        d1=d1
        d2=d2
        d3=d3
        d4=d4
top_day_trips=day_trips_df
top_day_trips

,Breakfast Area,Museum,Sight,Dining Area,Nightlife Area
0,Breakfast Area 2,Museum Bank Indonesia,Kota Tua,Dining Area 0,Nightlife Area 1
1,Breakfast Area 5,Museum Nasional Indonesia,Monumen Nasional (MONAS),Dining Area 2,Nightlife Area 5
2,Breakfast Area 1,Museum Satria Mandala,Kementerian Negara Koperasi dan UKM,Dining Area 1,Nightlife Area 3
3,Breakfast Area 4,Museum Sumpah Pemuda,Bank Indonesia,Dining Area 4,Nightlife Area 2
4,Breakfast Area 0,Museum Bank Mandiri,Bundaran Hotel Indonesia (Monumen Selamat Datang),Dining Area 5,Nightlife Area 4
5,Breakfast Area 3,Museum Kebangkitan Nasional,Patung Jenderal Sudirman,Dining Area 3,Nightlife Area 0


In [469]:
itinerary=top_day_trips.T
itinerary

,0,1,2,3,4,5
Breakfast Area,Breakfast Area 2,Breakfast Area 5,Breakfast Area 1,Breakfast Area 4,Breakfast Area 0,Breakfast Area 3
Museum,Museum Bank Indonesia,Museum Nasional Indonesia,Museum Satria Mandala,Museum Sumpah Pemuda,Museum Bank Mandiri,Museum Kebangkitan Nasional
Sight,Kota Tua,Monumen Nasional (MONAS),Kementerian Negara Koperasi dan UKM,Bank Indonesia,Bundaran Hotel Indonesia (Monumen Selamat Datang),Patung Jenderal Sudirman
Dining Area,Dining Area 0,Dining Area 2,Dining Area 1,Dining Area 4,Dining Area 5,Dining Area 3
Nightlife Area,Nightlife Area 1,Nightlife Area 5,Nightlife Area 3,Nightlife Area 2,Nightlife Area 4,Nightlife Area 0


In [470]:
itinerary_3=itinerary[0].append(itinerary[1]).append(itinerary[2])

itinerary_df=pd.DataFrame(itinerary_3)
itinerary_df.index=['Day 1','Day 1','Day 1','Day 1','Day 1','Day 2','Day 2','Day 2','Day 2','Day 2','Day 3','Day 3','Day 3','Day 3','Day 3',]
itinerary_df.columns=['Item Name']
itinerary_df=itinerary_df.join(day_areas_merged[['Item Name','Item Latitude','Item Longitude']].set_index('Item Name'), on='Item Name')
itinerary_df

,Item Name,Item Latitude,Item Longitude
Day 1,Breakfast Area 2,53.549519,10.037344
Day 1,Museum Bank Indonesia,-6.137120,106.813148
Day 1,Kota Tua,-6.135388,106.812949
Day 1,Dining Area 0,-6.154077,106.813941
Day 1,Nightlife Area 1,-6.147751,106.814530
Day 2,Breakfast Area 5,53.505420,9.986052
Day 2,Museum Nasional Indonesia,-6.176206,106.822854
Day 2,Monumen Nasional (MONAS),-6.175372,106.827192
Day 2,Dining Area 2,-6.159582,106.835583
Day 2,Nightlife Area 5,-6.154381,106.849596


3.6.9 Determine centroid of breakfast areas and nightlife areas (1st and last item of each itinerary should be close to hotel)

In [471]:
# set number of clusters
top_day_trips_kclusters = 1

breakfast_nightlife_top_day_trips=top_day_trips.loc[:,'Breakfast Area'].to_list() + top_day_trips.loc[:,'Nightlife Area'].to_list()
#day_areas_merged_2=day_areas_merged.loc[((day_areas_merged['Item Name'].isin(breakfast_top_day_trips)) or (day_areas_merged['Item Name'].isin(nightlife_top_day_trips))),:]
day_areas_merged_2=day_areas_merged.loc[day_areas_merged['Item Name'].isin(breakfast_nightlife_top_day_trips)==True,['Item Key','Item Latitude','Item Longitude']]


#hotel_area = top_day_trips.drop(['Venue','Venue Category', 'Venue ID', 'Venue Rating','Venue Key'], 1)

# run k-means clustering
top_day_trips_kmeans = KMeans(n_clusters=top_day_trips_kclusters, random_state=0).fit(day_areas_merged_2)

# generate hotel location
hotel_location=[top_day_trips_kmeans.cluster_centers_[0][1],top_day_trips_kmeans.cluster_centers_[0][2]]
print('The best hotel location is around coordinates '+str(hotel_location))

The best hotel location is around coordinates [23.681270255539854, 58.40432712162374]


3.6.10 Determnine the best hotel (central location for all 3 itineraries)

In [472]:
# Foursquare.com credentials and parameters
CLIENT_ID = 'R2B3BSMOAZU414BSOGLAYMB2MJPFUDT1PPEL1M0KZMD4RJFK' # your Foursquare ID
CLIENT_SECRET = '1YM0O41P05UKBSIDUTBL1G4ZYCAKEVRNQ4VFIO40THDU3EWW' # your Foursquare Secret
VERSION = '20201110' # Foursquare API version

categoryId='4bf58dd8d48988d1fa931735'

#Hotel
#4bf58dd8d48988d1fa931735

hotels_category_name_exclude=[]

radius=5000
LIMIT = 100 # A default Foursquare API limit value

# create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            hotel_location[0], 
            hotel_location[1], 
            VERSION, 
            radius,
            LIMIT,
            categoryId)
            
# make the GET request
hotels_json = requests.get(url).json()['response']['venues']
       
# return only relevant information for each nearby venue
hotels_list=[]


for v in hotels_json:
        try:
            venue_name=v['name']
        except:
            venue_name='no data'
        try:
            venue_lat=v['location']['lat']
        except:
            venue_lat='no data'
        try:
            venue_lng=v['location']['lng']
        except:
            venue_lng='no data'
        try:
            venue_lat=v['location']['lat']
        except:
            venue_lat='no data'
        try:
            category_name=v['categories'][0]['name']
        except:
            category_name='no data'
        try:
            venue_id=v['id']
        except:
            venue_id='no data'
        
        hotels_list.append([(
            venue_name, 
            venue_lat, 
            venue_lng, 
            category_name,
            venue_id)])

if len(hotels_list)==0:
    print('No hotels found within radius of {}m of the optimum hotel location'.format(radius))

else:
        hotels_df = pd.DataFrame([item for venue_list in hotels_list for item in venue_list])
        hotels_df.columns = ['Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Name','Venue ID']
        hotels_df.reset_index(drop=True, inplace=True)
        hotels_df=hotels_df[hotels_df['Venue Name'].isin(hotels_category_name_exclude)==False]
        print('Number of hotels in {} (latitude: {},longitude: {}): {}\n'.format(address,hotel_location[0],hotel_location[1],hotels_df.shape[0]))
        hotels_df

No hotels found within radius of 5000m of the optimum hotel location


In [473]:
hotels_df.head()

,Venue,Venue Latitude,Venue Longitude,Venue Name,Venue ID,Venue Rating
0,Sir Nikolai Hotel,53.546062,9.992378,Hotel,56693c4b498ef50133e2a32d,8.1
1,Innside Hamburg,53.545429,10.015054,Hotel,5952695775cb8c6a82345aaf,7.3
2,B&B Hotel Hamburg City-Ost,53.552420,10.028358,Hotel,57f370aecd107cfe93e1a40c,6.7
3,Moxy Hamburg City,53.552915,10.029087,Hotel,5f6e219b4cff341d503e6cf5,no rating
4,Holiday Inn Hamburg - Berliner Tor,53.551992,10.031498,Hotel,5c82d541a9e402002cdc711d,no rating


In [474]:
# retrieve ratings of hotels and append to hotels_df (Attention: venue/rating is a premium call!)
venue_id=hotels_df.loc[:,'Venue ID']
hotels_df.loc[:,'Venue Rating']=0

for i in range(len(venue_id)):
#    print('\nvenue_id [',i,']:',venue_id[i])
    venue_var=venue_id[i]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_var, CLIENT_ID, CLIENT_SECRET, VERSION)
    try:
#        print('Venue rating: ', requests.get(url).json()['response']['venue']['rating'])
        hotels_df.loc[i,'Venue Rating']=requests.get(url).json()['response']['venue']['rating']
    except:
#        print('This hotel has not been rated yet.')
        hotels_df.loc[i,'Venue Rating']='no rating'

hotels_df.head()

,Venue,Venue Latitude,Venue Longitude,Venue Name,Venue ID,Venue Rating
0,Sir Nikolai Hotel,53.546062,9.992378,Hotel,56693c4b498ef50133e2a32d,8.1
1,Innside Hamburg,53.545429,10.015054,Hotel,5952695775cb8c6a82345aaf,7.3
2,B&B Hotel Hamburg City-Ost,53.552420,10.028358,Hotel,57f370aecd107cfe93e1a40c,6.7
3,Moxy Hamburg City,53.552915,10.029087,Hotel,5f6e219b4cff341d503e6cf5,no rating
4,Holiday Inn Hamburg - Berliner Tor,53.551992,10.031498,Hotel,5c82d541a9e402002cdc711d,no rating


In [475]:
#Show top rated hotel near most convenient hotel location (hotels without rating dropped)
top_hotels_df=hotels_df.loc[((hotels_df['Venue Rating']!='no rating')),:].sort_values('Venue Rating', ascending=False)
top_hotels_df=top_hotels_df.head(1)
top_hotels_df

,Venue,Venue Latitude,Venue Longitude,Venue Name,Venue ID,Venue Rating
38,Superbude St. Pauli,53.560331,9.959941,Hostel,4f3e21b6e4b03dd023d9412e,9.2


In [476]:
hotel_lat=top_hotels_df.iloc[0,1]
hotel_lng=top_hotels_df.iloc[0,2]
hotel_name=top_hotels_df.iloc[0,0]
print('Hotel coordinates: '+str(hotel_lat)+', '+str(hotel_lng))

Hotel coordinates: 53.56033071877701, 9.959941098874191


3.6.11 Generate itineraries' path coordinates

In [477]:
# Route for day 1
route1=pd.DataFrame(data={'Item Name':hotel_name, 'Item Latitude':hotel_lat,'Item Longitude':hotel_lng},index=['Day 1'])
route_1=route1.append(itinerary_df[itinerary_df.index=='Day 1'])
route_1=route_1.append(route1)
route_1

,Item Name,Item Latitude,Item Longitude
Day 1,Superbude St. Pauli,53.560331,9.959941
Day 1,Breakfast Area 2,53.549519,10.037344
Day 1,Museum Bank Indonesia,-6.137120,106.813148
Day 1,Kota Tua,-6.135388,106.812949
Day 1,Dining Area 0,-6.154077,106.813941
Day 1,Nightlife Area 1,-6.147751,106.814530
Day 1,Superbude St. Pauli,53.560331,9.959941


In [478]:
# Route for day 2
route2=pd.DataFrame(data={'Item Name':hotel_name, 'Item Latitude':hotel_lat,'Item Longitude':hotel_lng},index=['Day 2'])
route_2=route2.append(itinerary_df[itinerary_df.index=='Day 2'])
route_2=route_2.append(route2)
route_2

,Item Name,Item Latitude,Item Longitude
Day 2,Superbude St. Pauli,53.560331,9.959941
Day 2,Breakfast Area 5,53.505420,9.986052
Day 2,Museum Nasional Indonesia,-6.176206,106.822854
Day 2,Monumen Nasional (MONAS),-6.175372,106.827192
Day 2,Dining Area 2,-6.159582,106.835583
Day 2,Nightlife Area 5,-6.154381,106.849596
Day 2,Superbude St. Pauli,53.560331,9.959941


In [479]:
# Route for day 3
route3=pd.DataFrame(data={'Item Name':hotel_name, 'Item Latitude':hotel_lat,'Item Longitude':hotel_lng},index=['Day 3'])
route_3=route3.append(itinerary_df[itinerary_df.index=='Day 3'])
route_3=route_3.append(route3)
route_3

,Item Name,Item Latitude,Item Longitude
Day 3,Superbude St. Pauli,53.560331,9.959941
Day 3,Breakfast Area 1,53.552170,9.993053
Day 3,Museum Satria Mandala,-6.230443,106.819408
Day 3,Kementerian Negara Koperasi dan UKM,-6.219321,106.831273
Day 3,Dining Area 1,-6.216916,106.825365
Day 3,Nightlife Area 3,-6.222659,106.833535
Day 3,Superbude St. Pauli,53.560331,9.959941


In [480]:
route_1_lat_lng=[]
for i in range(len(route_1)):
    route_1_lat_lng=route_1_lat_lng+[[route_1.iloc[i,1],route_1.iloc[i,2]]]
route_1_lat_lng

[[53.56033071877701, 9.959941098874191],
 [53.549519013337054, 10.03734380039381],
 [-6.137119907490385, 106.81314826011658],
 [-6.135388109075227, 106.8129492694242],
 [-6.154076934211658, 106.81394052236168],
 [-6.147750863033851, 106.81452984918408],
 [53.56033071877701, 9.959941098874191]]

In [481]:
route_2_lat_lng=[]
for i in range(len(route_2)):
    route_2_lat_lng=route_2_lat_lng+[[route_2.iloc[i,1],route_2.iloc[i,2]]]
route_2_lat_lng

[[53.56033071877701, 9.959941098874191],
 [53.505420384109705, 9.986052079564017],
 [-6.176205793009856, 106.82285363366826],
 [-6.175371641294809, 106.82719230651855],
 [-6.15958227637278, 106.83558308886202],
 [-6.154380939619149, 106.849596413382],
 [53.56033071877701, 9.959941098874191]]

In [482]:
route_3_lat_lng=[]
for i in range(len(route_3)):
    route_3_lat_lng=route_3_lat_lng+[[route_3.iloc[i,1],route_3.iloc[i,2]]]
route_3_lat_lng

[[53.56033071877701, 9.959941098874191],
 [53.55216969001092, 9.993053279929908],
 [-6.230443395151725, 106.81940767719075],
 [-6.219321220982749, 106.83127345517072],
 [-6.216915595060597, 106.82536471670005],
 [-6.2226594852533275, 106.83353482647635],
 [53.56033071877701, 9.959941098874191]]

####3.7 Visualize the itineraries on a map

In [483]:
from folium.features import DivIcon

In [484]:
# create map
map_dining_areas = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the restaurant clusters
restaurants_x = np.arange(restaurants_kclusters)
restaurants_ys = [i + restaurants_x + (i*restaurants_x)**2 for i in range(restaurants_kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(restaurants_ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
    
for area_restaurants_lat, area_restaurants_lon, area_restaurants_cluster in zip(top_dining_areas['Item Latitude'].unique(), top_dining_areas['Item Longitude'].unique(), top_dining_areas['Item Name'].unique()):
    area_restaurants_label = folium.Popup(' Dining Area ' + str(area_restaurants_cluster), parse_html=True)
    folium.CircleMarker(
        [area_restaurants_lat, area_restaurants_lon],
        radius=5,
        popup=area_restaurants_label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=1.0).add_to(map_dining_areas)   
    
# set color scheme for the nightlife clusters
bars_x = np.arange(bars_kclusters)
bars_ys = [i + bars_x + (i*bars_x)**2 for i in range(bars_kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(bars_ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for area_bars_lat, area_bars_lon, area_bars_cluster in zip(top_nightlife_areas['Item Latitude'].unique(), top_nightlife_areas['Item Longitude'].unique(), top_nightlife_areas['Item Name'].unique()):
    # Define marker label
    area_bars_label = folium.Popup(' Nightlife Area ' + str(area_bars_cluster), parse_html=True)

    # define square marker
    #icon_square = BeautifyIcon(icon_shape='rectangle-dot', border_color='blue', border_width=5,)
    
    # add marker to map
    folium.CircleMarker(
        [area_bars_lat, area_bars_lon],
        radius=5,
        popup=area_bars_label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=1.0).add_to(map_dining_areas)

for area_breakfast_lat, area_breakfast_lon, area_breakfast_cluster in zip(top_breakfast_areas['Item Latitude'].unique(), top_breakfast_areas['Item Longitude'].unique(), top_breakfast_areas['Item Name'].unique()):
    # Define marker label
    area_breakfast_label = folium.Popup(' Breakfast Area ' + str(area_breakfast_cluster), parse_html=True)

    # add marker to map
    folium.CircleMarker(
        [area_breakfast_lat, area_breakfast_lon],
        radius=5,
        popup=area_breakfast_label,
        color='orange',
        fill=True,
        fill_color='orange',
        fill_opacity=1.0).add_to(map_dining_areas)
        
for top_museums_lat, top_museums_lon, top_museums_cluster in zip(top_museums_df['Item Latitude'].unique(), top_museums_df['Item Longitude'].unique(), top_museums_df['Venue'].unique()):
    # Define marker label
    top_museums_label = folium.Popup(' Top Museum ' + str(top_museums_cluster), parse_html=True)
    
    # add marker to map
    folium.CircleMarker(
        [top_museums_lat, top_museums_lon],
        radius=5,
        popup=top_museums_label,
        color='yellow',
        fill=True,
        fill_color='yellow',
        fill_opacity=1.0).add_to(map_dining_areas)
    
for top_sights_lat, top_sights_lon, top_sights_cluster in zip(top_sights_df['Item Latitude'].unique(), top_sights_df['Item Longitude'].unique(), top_sights_df['Venue'].unique()):
    # Define marker label
    top_sights_label = folium.Popup(' Top Sight ' + str(top_sights_cluster), parse_html=True)
    
    # add marker to map
    folium.CircleMarker(
        [top_sights_lat, top_sights_lon],
        radius=5,
        popup=top_sights_label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=1.0).add_to(map_dining_areas)
        
# Show route and items of Day 1
for day_1_lat, day_1_lon, day_1_name, i in zip(itinerary_df.loc[itinerary_df.index=='Day 1', 'Item Latitude'], itinerary_df.loc[itinerary_df.index=='Day 1', 'Item Longitude'], itinerary_df.loc[itinerary_df.index=='Day 1', 'Item Name'], range(1,len(itinerary_df.loc[itinerary_df.index=='Day 1', 'Item Name'])+1)):
    # Define marker label
    day_1_label = folium.Popup('Day 1:' + str(day_1_name), parse_html=True)
    # add markers to map
    folium.CircleMarker(
        [day_1_lat, day_1_lon],
        radius=10,
        popup=day_1_label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=1.0
    ).add_to(map_dining_areas)
    folium.Marker(
        [day_1_lat, day_1_lon], 
        icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(10,4.5),
        html='<div style="font-size: 10px; color : white;text-align:center">'+str(i)+'</div>',
        )).add_to(map_dining_areas)
folium.PolyLine(route_1_lat_lng, color='blue').add_to(map_dining_areas)

# Show route and items of Day 2
for day_2_lat, day_2_lon, day_2_name, i in zip(itinerary_df.loc[itinerary_df.index=='Day 2', 'Item Latitude'], itinerary_df.loc[itinerary_df.index=='Day 2', 'Item Longitude'], itinerary_df.loc[itinerary_df.index=='Day 2', 'Item Name'], range(1,len(itinerary_df.loc[itinerary_df.index=='Day 2', 'Item Name'])+1)):
    # Define marker label
    day_2_label = folium.Popup('Day 2:' + str(day_2_name), parse_html=True)
    # add markers to map
    folium.CircleMarker(
        [day_2_lat, day_2_lon],
        radius=10,
        popup=day_2_label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=1.0
    ).add_to(map_dining_areas)
    folium.Marker(
        [day_2_lat, day_2_lon], 
        icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(10,4.5),
        html='<div style="font-size: 10px; color : white;text-align:center">'+str(i)+'</div>',
        )).add_to(map_dining_areas)
folium.PolyLine(route_2_lat_lng, color='green').add_to(map_dining_areas)

# Show route and items of Day 3
for day_3_lat, day_3_lon, day_3_name, i in zip(itinerary_df.loc[itinerary_df.index=='Day 3', 'Item Latitude'], itinerary_df.loc[itinerary_df.index=='Day 3', 'Item Longitude'], itinerary_df.loc[itinerary_df.index=='Day 3', 'Item Name'], range(1,len(itinerary_df.loc[itinerary_df.index=='Day 3', 'Item Name'])+1)):
    # Define marker label
    day_3_label = folium.Popup('Day 3:' + str(day_3_name), parse_html=True)
    # add markers to map
    folium.CircleMarker(
        [day_3_lat, day_3_lon],
        radius=10,
        popup=day_3_label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=1.0
    ).add_to(map_dining_areas)
    folium.Marker(
        [day_3_lat, day_3_lon], 
        icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(10,4.5),
        html='<div style="font-size: 10px; color : white;text-align:center">'+str(i)+'</div>',
        )).add_to(map_dining_areas)
folium.PolyLine(route_3_lat_lng, color='red').add_to(map_dining_areas)
    
# add hotel marker to map
folium.CircleMarker(
        [hotel_lat, hotel_lng],
        radius=20,
        popup='Hotel Ritz',
        tooltip='Hotel',
        color='magenta',
        fill=True,
        fill_color='magenta',
        fill_opacity=1.0,
).add_to(map_dining_areas)
folium.Marker(
        [hotel_lat, hotel_lng], 
        icon=DivIcon(
        icon_size=(40,40),
        icon_anchor=(20,9),
        html='<div style="font-size: 12px; color : white;text-align:center">Hotel</div>',
        )).add_to(map_dining_areas)

map_dining_areas